## Papermill Sampler Operation

__Description__: Tool to calculate the mean flow frequency curve, specify a range of annual exceedance probabilities with corresponding weights, and assign those weights to discharge events. For additional details, see the [Sampler Operation Wiki](https://github.com/Dewberry/probmod-tools/wiki/Sampler-Operation).

__Input__: [HEC-SSP](https://www.hec.usace.army.mil/software/hec-ssp/) .rpt files containing flow frequency data for a specific USGS Stream Gage calculated at a range of confidence limits, and the following notebooks: `SSP_to_Mean_Curve`, `Stratified_Sampler`, and `Make_Production_Run_List`.

__Output__: Excel workbook with a sheet for the mean flow frequency curve table, the annual exceedance probability (AEP) vs weight table, and the discharge verses weight table.


---

### Load libraries:

In [1]:
import sys
sys.path.append('../../core')
from nb_libraries import*
from fluvial_hydro import*

### Specify the location to save the outputs:

In [2]:
output_path = r'C:\Users\sputnam\Desktop\Outputs'

if os.path.isdir(output_path)==False: #If the desired path does not exist, create it.
    os.mkdir(output_path)    

### 1) Run SSP_to_Mean_Curve Notebook:

##### Specify: 

In [3]:
gage_ID = '01651000' #The USGS Station ID 

root_dir = r'C:\Users\sputnam\Desktop\Inputs' #The path where the SSP files can be found

version = 2_2 #The version of HEC-SSP used to calculate the .rpt files. Either 2.1 or 2.2, specified as 2_1 or 2_2 here.

min_val = 2.0    #The minimum recurrance interval

max_val = 3000.0 #The maximum recurrance interval

nbin = 100 #The number of recurrance intervals/AEPs

##### Run:

In [4]:
out_nb1 = os.path.join(output_path,'SSP_to_Mean_Curve_{0}.ipynb'.format(gage_ID))
nb1 = pm.execute_notebook('SSP_to_Mean_Curve.ipynb',
                          out_nb1, 
                          parameters={'root_dir':root_dir, 'gage_ID':gage_ID, 'version':version})

##### Display results:

In [5]:
nb1 = sb.read_notebook(out_nb1)

mean_curve = format_mean_curve(nb1.scraps['mean_curve'][1])

display(mean_curve.head(5))

,Q_Mean_cfs,Q_Median_cfs
AEP,,
0.90,1677.026488,1674.7
0.50,3374.838485,3338.6
0.20,5932.199123,5821.0
0.10,8248.414233,8056.3
0.05,11160.171869,10731.8


### 2) Run Stratified_Sampler Notebook:

##### Run:

In [6]:
out_nb2 = os.path.join(output_path,'Stratified_Sampler_{0}.ipynb'.format(gage_ID))
nb2 = pm.execute_notebook("Stratified_Sampler.ipynb", out_nb2, 
                        parameters={'root_dir':root_dir, 'gage_ID':gage_ID, 'min_val':min_val, 'max_val':max_val, 'nbin':nbin})

##### Display results:

In [7]:
nb2 = sb.read_notebook(out_nb2)

AEP_Weights = pd.DataFrame.from_dict(nb2.scraps['AEP_Weights'][1])

display(AEP_Weights.head(5))

,AEP,Weight
E0001,0.482369,0.035594
E0002,0.448352,0.032774
E0003,0.416733,0.030463
E0004,0.387344,0.028315
E0005,0.360028,0.026318


### 3) Run Make_Production_Run_List Notebook: 

##### Run:

In [8]:
out_nb3 = os.path.join(output_path,'Make_Production_Run_List_{0}.ipynb'.format(gage_ID))
nb3 = pm.execute_notebook("Make_Production_Run_List.ipynb", out_nb3, 
                        parameters={'root_dir':root_dir, 'gage_ID':gage_ID, 'mean_curve':mean_curve.to_dict(),'AEP_Weights':AEP_Weights.to_dict()})

##### Display results:

In [9]:
nb3 = sb.read_notebook(out_nb3)

ProRunList = pd.DataFrame.from_dict(nb3.scraps['ProRunList'][1])

display(ProRunList.head(5))

,AEP,Q_Mean_cfs,Q_Median_cfs,RI,Weight
E0001,0.482369,3476.322534,3437.526589,2.073100,0.035594
E0002,0.448352,3681.632218,3637.536355,2.230391,0.032774
E0003,0.416733,3885.535115,3836.016550,2.399617,0.030463
E0004,0.387344,4088.541263,4033.474737,2.581683,0.028315
E0005,0.360028,4291.058977,4230.317069,2.777562,0.026318


### 4) Save results:

##### Save to Excel:

In [10]:
with pd.ExcelWriter(os.path.join(output_path,'Sampler_Operation_{0}.xlsx'.format(gage_ID))) as writer:
    mean_curve.to_excel(writer, sheet_name='Mean Curve') 
    
    AEP_Weights.to_excel(writer, sheet_name='AEP Weights')
    
    ProRunList.to_excel(writer, sheet_name='Production Run List')
    
! jupyter nbconvert $out_nb1 --log-level WARN
! jupyter nbconvert $out_nb2 --log-level WARN
! jupyter nbconvert $out_nb3 --log-level WARN    

c:\users\sputnam\appdata\local\continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able to be represented.
  mimetypes=output.keys())
c:\users\sputnam\appdata\local\continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able to be represented.
  mimetypes=output.keys())
c:\users\sputnam\appdata\local\continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able to be represented.
  mimetypes=output.keys())


# End